# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords


from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report

from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier

from sklearn.base import BaseEstimator, TransformerMixin


import pickle

[nltk_data] Downloading package punkt to /Users/vikram/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/vikram/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vikram/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/vikram/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [6]:
# load data from database
engine = create_engine('sqlite:///DisasterData.db')
df = pd.read_sql_table('DisasterData', engine)
X = df['message']  #['id','message','original','genre']
Y = df.drop(['id','message','original','genre'], axis = 1)

In [7]:
Y['related'].value_counts()

1.0    19914
0.0     6101
2.0      192
Name: related, dtype: int64

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9]"," ", text)
    words = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    stop_words = stopwords.words("english")

    #clean_words = []
    #for word in words:
        #clean_words.append(lemmatizer.lemmatize(word).lower().strip())
    clean_words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
                           
    return clean_words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
pipeline = Pipeline([
    ('vectorizer', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))    
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

> Found that there are null values, which is why the fit pipeline cell below was not working. As an intial step (may go back and improve on later), let's just drop these few ~100 rows where there are nan values. This makes sense because if we are trying to predict the values of for these columns based of the message, and all these values are Null, they don't give us anything in terms of information


In [8]:
df_clean = df[np.isfinite(df['related'])]
X = df_clean['message']  #['id','message','original','genre']
Y = df_clean.drop(['id','message','original','genre'], axis = 1)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X,Y)
pipeline.fit(X_train, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The de

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
y_pred = pipeline.predict(X_test)

In [9]:
# y_pred is not a data frame, let's make it into a data frame so we can see things more easily
df_ypred = pd.DataFrame(y_pred, columns = y_test.columns)
df_ypred.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# Printing out "precision", "recall", and "f1_scores" for each column
for col in y_test.columns:
    print(col)
    print(classification_report(y_test[col].values, df_ypred[col]))

related
              precision    recall  f1-score   support

         0.0       0.34      0.15      0.21      1580
         1.0       0.76      0.91      0.83      4919
         2.0       0.06      0.02      0.03        53

   micro avg       0.72      0.72      0.72      6552
   macro avg       0.39      0.36      0.35      6552
weighted avg       0.66      0.72      0.67      6552

request
              precision    recall  f1-score   support

         0.0       0.84      0.97      0.90      5467
         1.0       0.34      0.09      0.14      1085

   micro avg       0.82      0.82      0.82      6552
   macro avg       0.59      0.53      0.52      6552
weighted avg       0.76      0.82      0.77      6552

offer
              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00      6519
         1.0       0.00      0.00      0.00        33

   micro avg       0.99      0.99      0.99      6552
   macro avg       0.50      0.50      0.50      655

/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

         0.0       0.95      1.00      0.97      6238
         1.0       0.00      0.00      0.00       314

   micro avg       0.95      0.95      0.95      6552
   macro avg       0.48      0.50      0.49      6552
weighted avg       0.91      0.95      0.93      6552

search_and_rescue
              precision    recall  f1-score   support

         0.0       0.97      1.00      0.99      6366
         1.0       0.00      0.00      0.00       186

   micro avg       0.97      0.97      0.97      6552
   macro avg       0.49      0.50      0.49      6552
weighted avg       0.94      0.97      0.96      6552

security
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      6428
         1.0       0.00      0.00      0.00       124

   micro avg       0.98      0.98      0.98      6552
   macro avg       0.49      0.50      0.50      6552
weighted avg       0.96      0.98      0.97    

              precision    recall  f1-score   support

         0.0       0.91      0.99      0.95      5950
         1.0       0.35      0.04      0.08       602

   micro avg       0.90      0.90      0.90      6552
   macro avg       0.63      0.52      0.51      6552
weighted avg       0.86      0.90      0.87      6552

fire
              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      6473
         1.0       0.00      0.00      0.00        79

   micro avg       0.99      0.99      0.99      6552
   macro avg       0.49      0.50      0.50      6552
weighted avg       0.98      0.99      0.98      6552

earthquake
              precision    recall  f1-score   support

         0.0       0.92      0.99      0.95      5916
         1.0       0.65      0.17      0.28       636

   micro avg       0.91      0.91      0.91      6552
   macro avg       0.78      0.58      0.61      6552
weighted avg       0.89      0.91      0.89      6552

col

### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
pipeline.get_params()

{'memory': None,
 'steps': [('vectorizer',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x10c8057b8>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
               oob_score=False, random_s

### Using the parameters listed above I can see what parameters yeild the best results. In order to do this, I first structured the code into a model similar to the model shown in the lectures. This involved building a model and having a main() function call that loads the data, fits it, and then shows us the best parameters.

>The starting accuracy was about <b>92.3%</b> so we are hoping to achieve something a little bit higher than this through some trial and error, as well as following the hints listed below

In [21]:
# Function to build the pipeline and create grid search
def build_model():
    
    pipeline = Pipeline([
        ('vectorizer', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))    
    ])
    
    # specify parameters for grid search
    parameters = {'vectorizer__max_df': (0.4, .8),
                  'vectorizer__max_features': [None],
                  'tfidf__use_idf': [False],
                  'vectorizer__lowercase': [True],
                  'clf__estimator__n_estimators': [1, 5, 50]
                 }

    # create grid search object
    cv = GridSearchCV(pipeline, param_grid=parameters, verbose=2)
    
    return cv

In [102]:
# Function to read in the data and return X and y to be used for the training and test sets
def load_data():
    engine = create_engine('sqlite:///DisasterData.db')
    df = pd.read_sql_table('DisasterData', engine)
    df_clean = df[np.isfinite(df['related'])]
    X = df_clean['message']  #['id','message','original','genre']
    Y = df_clean.drop(['id','message','original','genre'], axis = 1)
    return X, Y

# Function used by the main function to display the data in a readable way
def display_results(cv, y_test, y_pred):
    labels = np.unique(y_pred)
    y_pred = y_pred.reset_index(drop=True)
    y_test = y_test.reset_index(drop=True)
    accuracy = (y_pred == y_test).mean().mean()

    print("Labels:", labels)
    print("Accuracy:", accuracy)
    print("\nBest Parameters:", cv.best_params_)

# Main function call. Process the data and displays results
def main():
    X, y = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    model = build_model()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    df_ypred = pd.DataFrame(y_pred, columns = y_test.columns)
    
    display_results(model, y_test, df_ypred)


In [103]:
# This function takes a while to run. Added a 'verbose = 2' so that we can track the progress of the main
# function as it is running. As I tweak parameters I will continue to come back here and run this main function call
main()

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] clf__estimator__n_estimators=1, tfidf__use_idf=False, vectorizer__lowercase=True, vectorizer__max_df=0.4, vectorizer__max_features=None 
[CV]  clf__estimator__n_estimators=1, tfidf__use_idf=False, vectorizer__lowercase=True, vectorizer__max_df=0.4, vectorizer__max_features=None, total=  18.1s
[CV] clf__estimator__n_estimators=1, tfidf__use_idf=False, vectorizer__lowercase=True, vectorizer__max_df=0.4, vectorizer__max_features=None 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   28.2s remaining:    0.0s


[CV]  clf__estimator__n_estimators=1, tfidf__use_idf=False, vectorizer__lowercase=True, vectorizer__max_df=0.4, vectorizer__max_features=None, total=  17.7s
[CV] clf__estimator__n_estimators=1, tfidf__use_idf=False, vectorizer__lowercase=True, vectorizer__max_df=0.4, vectorizer__max_features=None 
[CV]  clf__estimator__n_estimators=1, tfidf__use_idf=False, vectorizer__lowercase=True, vectorizer__max_df=0.4, vectorizer__max_features=None, total=  17.9s
[CV] clf__estimator__n_estimators=1, tfidf__use_idf=False, vectorizer__lowercase=True, vectorizer__max_df=0.8, vectorizer__max_features=None 
[CV]  clf__estimator__n_estimators=1, tfidf__use_idf=False, vectorizer__lowercase=True, vectorizer__max_df=0.8, vectorizer__max_features=None, total=  18.0s
[CV] clf__estimator__n_estimators=1, tfidf__use_idf=False, vectorizer__lowercase=True, vectorizer__max_df=0.8, vectorizer__max_features=None 
[CV]  clf__estimator__n_estimators=1, tfidf__use_idf=False, vectorizer__lowercase=True, vectorizer__max

[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed: 37.4min finished


Labels: [0. 1. 2.]
Accuracy: 0.9244717473884142

Best Parameters: {'clf__estimator__n_estimators': 50, 'tfidf__use_idf': False, 'vectorizer__lowercase': True, 'vectorizer__max_df': 0.8, 'vectorizer__max_features': None}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

> I answer this in the main function call above. By adding in the `accuracy` in `display_results` we see how the results improved from 92.3%

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

* Noticed that for the first column, the possible values are 0, 1, and a few 2's. What does this mean here and what is the best way to handle the 2's. Are they errors/should be 1's??

In [123]:
# Function to build the pipeline and create grid search
def build_better_model():
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('starting_verb', StartingVerbExtracter()),
            
            ('text_pipeline', Pipeline([
                ('vectorizer', CountVectorizer(tokenizer = tokenize)),
                ('tfidf', TfidfTransformer())
                ])
            ),
        ])),
        
        ('clf', MultiOutputClassifier(RandomForestClassifier()))    
    ])
    
    return pipeline

In [13]:
def new_model_pipeline():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('verb_pipeline', Pipeline([
                ('starting_verb', StartingVerbExtractor()),
                ('caster', ArrayCaster())
            ]))
           
        ])),

        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])

    return pipeline

In [19]:
def build_much_better_model():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('verb_pipeline', Pipeline([
                ('starting_verb', StartingVerbExtractor()),
                ('caster', ArrayCaster())
            ]))
           
        ])),

        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])

    
    parameters = {
    'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
    'features__text_pipeline__vect__max_df': (0.75, 1.0),
    'features__text_pipeline__vect__max_features': (None, 5000),
    #'features__text_pipeline__tfidf__use_idf': (True, False),
    }
    
    # create grid search object
    cv = GridSearchCV(pipeline, param_grid=parameters, verbose=2)
    return cv
    
    
    #return pipeline

In [20]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    
    def extract_starting_verb(self, text):
        '''
        Tokenizes text into sentances and checks if the first word is a verb
        Input:
            Text: Message to be tokenized by extract_starting_verb function
        Output:
            Boolean value for first word (True if a verb, False if not)
        '''
        sentences = nltk.sent_tokenize(text)
        for sentence in sentences:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            if len(pos_tags) > 0:
                first_word, first_type = pos_tags[0]
                if first_type in ['VB', 'VBP', 'VBG', 'VBN', 'VBP', 'VBZ'] or first_word == 'RT':
                    return True
        return False
        
    def transform(self, X):
        tagged_entries = pd.Series(X).apply(self.extract_starting_verb) # where the actual feature extraction happens
        return tagged_entries  

    def fit(self, X, y=None):
        return self  # generally does nothing
    

In [21]:
class ArrayCaster(BaseEstimator, TransformerMixin):
  
    def fit(self, x, y=None):
        return self

    def transform(self, data):
        return np.transpose(np.matrix(data))

In [22]:
newnew_pipeline = build_much_better_model()
newnew_pipeline.fit(X_train, y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__max_features=None, features__text_pipeline__vect__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__max_features=None, features__text_pipeline__vect__ngram_range=(1, 1), total= 1.9min
[CV] features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__max_features=None, features__text_pipeline__vect__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.6min remaining:    0.0s


[CV]  features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__max_features=None, features__text_pipeline__vect__ngram_range=(1, 1), total= 1.9min
[CV] features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__max_features=None, features__text_pipeline__vect__ngram_range=(1, 1) 
[CV]  features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__max_features=None, features__text_pipeline__vect__ngram_range=(1, 1), total= 1.9min
[CV] features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__max_features=None, features__text_pipeline__vect__ngram_range=(1, 2) 
[CV]  features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__max_features=None, features__text_pipeline__vect__ngram_range=(1, 2), total= 3.4min
[CV] features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__max_features=None, features__text_pipeline__vect__ngram_range=(1, 2) 
[CV]  features__text_pipeline__vect__max_df=0.75, featu

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed: 71.9min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=None,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, ma...or=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)), 'features__text_pipeline__vect__max_df': (0.75, 1.0), 'features__text_pipeline__vect__max_features': (None, 5000)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [46]:
new_pipeline = new_model_pipeline()
new_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=None,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, ma...or=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=None))])

In [23]:
y_pred = newnew_pipeline.predict(X_test)
(y_pred == y_test).mean().mean()

0.9262608533441868

In [121]:
# Function to read in the data and return X and y to be used for the training and test sets
def load_data():
    engine = create_engine('sqlite:///DisasterData.db')
    df = pd.read_sql_table('DisasterData', engine)
    df_clean = df[np.isfinite(df['related'])]
    X = df_clean['message']  #['id','message','original','genre']
    Y = df_clean.drop(['id','message','original','genre'], axis = 1)
    return X, Y

# Function used by the main function to display the data in a readable way
def display_results(cv, y_test, y_pred):
    labels = np.unique(y_pred)
    y_pred = y_pred.reset_index(drop=True)
    y_test = y_test.reset_index(drop=True)
    accuracy = (y_pred == y_test).mean().mean()

    print("Labels:", labels)
    print("Accuracy:", accuracy)
    print("\nBest Parameters:", cv.best_params_)

# Main function call. Process the data and displays results
def main():
    X, y = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    model = build_better_model()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    df_ypred = pd.DataFrame(y_pred, columns = y_test.columns)
    
    display_results(model, y_test, df_ypred)

In [122]:
main()

IndexError: list index out of range

In [118]:
y_train.shape

(19655, 36)

In [ ]:
#Y['related']=Y['related'].map(lambda x: 1 if x == 2 else x)

### 9. Export your model as a pickle file

In [ ]:
# saving
file_name = 'disasterresponse.sav'
pickle.dump(model, open(file_name, 'wb'))
 

In [ ]:
# Try and load the saved model to 
saved_model = pickle.load(open(filename, 'rb'))
findings = saved_model.score(X_test, y_test)
print(findings)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.